In [ ]:
from dataclasses import dataclass
from typing import Optional
@dataclass
class Elephant:
    name:    str = '大象'
    weight:  int = 9999
    color:   Optional[str] = '粉色'
    contant: str = '冰箱'
   
    def action_1(self):
        print(f"打开{self.contant}")
    def action_2(self):
        print(f"把{self.color}kg的{self.name}放进{self.contant}")
    def action_3(self):
        print(f"关闭{self.contant}")

a = Elephant()
for i in range(1,4):
    s = f"action_{i}"
    getattr(a,s)()

    

打开冰箱
把粉色kg的大象放进冰箱
关闭冰箱
